# Análisis Exploratorio de Datos 

In [ ]:
import subprocess, sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))
for pkg in ['pandas', 'numpy', 'plotly', 'seaborn', 'matplotlib']:
    try: __import__(pkg)
    except: subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
print(f' ROOT: {PROJECT_ROOT}')

✓ ROOT: c:\Proyecto_Enfermedades_Alto_Costo completo\Proyecto_Enfermedades_Alto_Costo completo


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
df = pd.read_parquet(DATA_PROCESSED / 'raw_clean.parquet')
df_dict = pd.read_parquet(DATA_PROCESSED / 'dictionary.parquet')
print(f' Dataset: {df.shape}')

✓ Dataset: (42644, 27)


## 1. Distribución de la Variable Objetivo

In [ ]:
target_col = 'GRUPO'
target_counts = df[target_col].value_counts()
fig = px.bar(x=target_counts.index, y=target_counts.values, 
             title='Distribución de Enfermedades de Alto Costo',
             labels={'x': 'Enfermedad', 'y': 'Frecuencia'},
             color=target_counts.values, color_continuous_scale='Reds')
fig.update_layout(showlegend=False, height=400)
fig.show()
print(" Distribución altamente desbalanceada con CANCER como clase dominante.")

📊 Distribución altamente desbalanceada con CANCER como clase dominante.


## 2. Variables Numéricas - Distribuciones

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
fig = make_subplots(rows=len(numeric_cols[:6])//2, cols=2, subplot_titles=numeric_cols[:6])
for i, col in enumerate(numeric_cols[:6], 1):
    row = (i-1)//2 + 1
    col_pos = (i-1)%2 + 1
    fig.add_trace(go.Histogram(x=df[col], name=col, showlegend=False), row=row, col=col_pos)
fig.update_layout(height=800, title_text="Distribuciones de Variables Numéricas")
fig.show()
print(" Se observan diferentes distribuciones: algunas simétricas, otras sesgadas.")

📊 Se observan diferentes distribuciones: algunas simétricas, otras sesgadas.


## 3. Análisis Geográfico

In [ ]:
if 'Nom Dep' in df.columns:
    dept_counts = df['Nom Dep'].value_counts().head(10)
    fig = px.bar(x=dept_counts.values, y=dept_counts.index, orientation='h',
                 title='Top 10 Departamentos por Número de Defunciones',
                 labels={'x': 'Defunciones', 'y': 'Departamento'})
    fig.show()
    print(" Los departamentos con mayor población muestran más casos (sesgo poblacional).")
else:
    print("Variable geográfica no disponible")

📊 Los departamentos con mayor población muestran más casos (sesgo poblacional).


## 4. Análisis Temporal

In [ ]:
if 'MES' in df.columns:
    mes_counts = df['MES'].value_counts().sort_index()
    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    fig = px.line(x=meses, y=mes_counts.values, title='Distribución de Defunciones por Mes',
                  labels={'x': 'Mes', 'y': 'Defunciones'})
    fig.update_traces(mode='lines+markers')
    fig.show()
    print(" Patrón temporal relativamente estable a lo largo del año.")

📊 Patrón temporal relativamente estable a lo largo del año.


## 5. Variables Demográficas

In [ ]:
if 'SEXO' in df.columns:
    sexo_grupo = df.groupby(['SEXO', target_col]).size().reset_index(name='count')
    fig = px.bar(sexo_grupo, x='SEXO', y='count', color=target_col,
                 title='Distribución de Enfermedades por Sexo', barmode='group')
    fig.show()
    print(" Distribución de enfermedades varía según sexo, CANCER predomina en ambos.")

📊 Distribución de enfermedades varía según sexo, CANCER predomina en ambos.


## 6. Correlaciones - Variables Numéricas

In [ ]:
numeric_data = df[numeric_cols].select_dtypes(include=[np.number])
corr_matrix = numeric_data.corr()
fig = px.imshow(corr_matrix, title='Matriz de Correlación',
                color_continuous_scale='RdBu_r', aspect='auto')
fig.show()
print(" Correlaciones bajas/moderadas entre variables, sin colinealidad severa.")

📊 Correlaciones bajas/moderadas entre variables, sin colinealidad severa.


## 7. Análisis de Outliers

In [ ]:
fig = go.Figure()
for col in numeric_cols[:5]:
    fig.add_trace(go.Box(y=df[col], name=col))
fig.update_layout(title='Boxplots - Detección de Outliers', height=500)
fig.show()
print(" Se detectan outliers en varias variables. No se eliminan para preservar información real.")

📊 Se detectan outliers en varias variables. No se eliminan para preservar información real.


## 8. Relación Variables con Target

In [ ]:
if 'GRU_ED2' in df.columns:
    edad_grupo = df.groupby(['GRU_ED2', target_col]).size().reset_index(name='count')
    fig = px.bar(edad_grupo, x='GRU_ED2', y='count', color=target_col,
                 title='Distribución de Enfermedades por Grupo de Edad', barmode='stack')
    fig.show()
    print(" Patrones etarios: CANCER más frecuente en grupos de mayor edad.")

📊 Patrones etarios: CANCER más frecuente en grupos de mayor edad.


Análisis: Se verifico el dataset procesado (42644, 27) y se observo patrones temporales relativamente estables a lo largo del año; distribución de enfermedades altamente desbalanceada con CANCER como clase dominante y diferencias marcadas por sexo y grupos de edad. Las correlaciones entre variables son bajas/moderadas.

Conclusiones: El EDA identifica a CANCER como el predictor dominante.